In [7]:
import pandas as pd
import sys
sys.path.append('../')
from machinelearning.mlpipeline import MLPipelines
from machinelearning.mlexplain import MLExplainer
from dataloader.eda import DataExplorer

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
csv_dir = '../data/epic_ce_ms.csv'
label = 'group'

mlpipe = MLPipelines(label=label, csv_dir=csv_dir)
f = mlpipe.feature_selection(method='mrmr', num_features=20, inner_method='chi2')

Label mapping: {0: 'Factor1:CASE', 1: 'Factor1:CONTROL'}
There is no selected classifier.


In [15]:
mlpipe.nested_cv()

2024-06-28 18:54:48.138778: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-28 18:54:48.139794: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-28 18:54:48.355412: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 18:54:48.357976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been r

<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object><IPython.core.display.HTML object>

<IPython.core.display.HTML object>


KeyboardInterrupt: 

In [ ]:
mlpipe.feature_selection(method='mrmr', num_features=20, inner_method='chi2')
# mlpipe.X=mlpipe.X.fillna(0)
params = {
    'n_estimators': [100],
}
model, results = mlpipe.random_search(estimator_name='RandomForestClassifier', parameter_grid=params, n_iter=3)